In [1]:
import azureml.core
from azureml.core import Workspace
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Experiment, RunConfiguration, ScriptRunConfig ,Environment

In [ ]:
# Check the Azure ML SDK version (optional but recommended)
print("Azure ML SDK Version: ", azureml.core.VERSION)

In [28]:
ws = Workspace.from_config()

In [ ]:
ws.datasets

In [3]:
compute_lists = ws.compute_targets

In [29]:
mlops_01 = ComputeTarget(ws,'MLOps-01')

In [33]:
mlops_01.delete()

False

In [4]:
type(compute_lists)

dict

In [13]:
compute_lists.items()

dict_items([('MLOps-01', AmlCompute(workspace=Workspace.create(name='machine-learning', subscription_id='72300789-c223-4047-bfd5-2859014ad9f7', resource_group='machinelearning-rg'), name=MLOps-01, id=/subscriptions/72300789-c223-4047-bfd5-2859014ad9f7/resourceGroups/machinelearning-rg/providers/Microsoft.MachineLearningServices/workspaces/machine-learning/computes/MLOps-01, type=AmlCompute, provisioning_state=Succeeded, location=eastasia, tags={}))])

In [16]:
import json
json.dumps(str(compute_lists),indent=4)

'"{\'MLOps-01\': AmlCompute(workspace=Workspace.create(name=\'machine-learning\', subscription_id=\'72300789-c223-4047-bfd5-2859014ad9f7\', resource_group=\'machinelearning-rg\'), name=MLOps-01, id=/subscriptions/72300789-c223-4047-bfd5-2859014ad9f7/resourceGroups/machinelearning-rg/providers/Microsoft.MachineLearningServices/workspaces/machine-learning/computes/MLOps-01, type=AmlCompute, provisioning_state=Succeeded, location=eastasia, tags={})}"'

In [19]:
from pprint import pprint
pprint(json.dumps(str(compute_lists),indent=4))

('"{\'MLOps-01\': '
 "AmlCompute(workspace=Workspace.create(name='machine-learning', "
 "subscription_id='72300789-c223-4047-bfd5-2859014ad9f7', "
 "resource_group='machinelearning-rg'), name=MLOps-01, "
 'id=/subscriptions/72300789-c223-4047-bfd5-2859014ad9f7/resourceGroups/machinelearning-rg/providers/Microsoft.MachineLearningServices/workspaces/machine-learning/computes/MLOps-01, '
 'type=AmlCompute, provisioning_state=Succeeded, location=eastasia, tags={})}"')


In [22]:
pprint(str(compute_lists))

("{'MLOps-01': AmlCompute(workspace=Workspace.create(name='machine-learning', "
 "subscription_id='72300789-c223-4047-bfd5-2859014ad9f7', "
 "resource_group='machinelearning-rg'), name=MLOps-01, "
 'id=/subscriptions/72300789-c223-4047-bfd5-2859014ad9f7/resourceGroups/machinelearning-rg/providers/Microsoft.MachineLearningServices/workspaces/machine-learning/computes/MLOps-01, '
 'type=AmlCompute, provisioning_state=Succeeded, location=eastasia, tags={})}')



## Get the list of Enviornment available



In [ ]:
# Load your Azure ML workspace
ws = Workspace.from_config()

# Get a list of available curated environments
envs = Environment.list(workspace=ws)

# Print the names of available curated environments
for env_name in envs:
    print(env_name)

### Create an environemnt from the requirements txt

In [ ]:
#Create an envionment from requirements.txt file
env = Environment.from_pip_requirements(name='mlops_01_env_from_pip_local',file_path='./requirements.txt',)
# Register the enviorment to the workspace
env.register(ws) 
#build the env
env.build(ws)

# Lets Start with an Experiment

In [ ]:
from azureml.core import Experiment, RunConfiguration, ScriptRunConfig ,Environment

# Load your Azure ML workspace from the config file
ws = Workspace.from_config()

# Create a new experiment
experiment_name = "iris-experiment"
experiment = Experiment(workspace=ws, name=experiment_name)



# Create A Run Configuration
run_config = RunConfiguration()
run_config.target = "MLOps-01"  #The compute_targets available 
run_config.environment = ws.environments['mlops_01_env_from_pip_local']



# Specify the output folder where the model and results will be saved
output_folder = "./outputs"  # Change this to your desired location


# Specify the script to run (train_01.py in this case)
script_run_config = ScriptRunConfig(source_directory=".", script="train_01.py", run_config=run_config,arguments=["--output_folder", output_folder])


# Submit the experiment
run = experiment.submit(config=script_run_config)
run.wait_for_completion(show_output=True)
